### Импортируем библиотеки, настраиваем отображение таблиц

In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.options.display.max_columns = 100

### Загружаем данные 20 и 21 г

In [3]:
file_20 = 'C:/Users/lazarevnv/Desktop///Sales_from_01.01.2020_to_31.12.2020_on_18.01.2022_14_43_30.txt'
file_21 = 'C:/Users/lazarevnv/Desktop////Sales_from_01.01.2021_to_31.12.2021_on_18.01.2022_15_04_15.txt'

In [4]:
df_20 = pd.read_csv(file_20, sep='\t', decimal=',', encoding = 'cp1251')

In [5]:
df_21 = pd.read_csv(file_21, sep='\t', encoding = 'cp1251', decimal=',')

In [6]:
df_20.shape, df_21.shape

((1866482, 12), (1859143, 12))

In [8]:
# df_20.info()

In [52]:
# df_21.info()

In [12]:
# df_20.head(2)

In [11]:
# df_21.head(2)

In [13]:
df = pd.concat((df_20, df_21), axis = 0)

In [18]:
# df['Distributor'].unique()

In [14]:
df.shape

(3725625, 12)

In [16]:
# df[['Год', 'Месяц', 'Сумма отгрузки, с НДС, руб']]\
#     .groupby(['Год', 'Месяц']).sum()

### Сверка с данными других отчётов (например SQL), анализ в случае спорных моментов

In [22]:
# df.loc[((df['Distributor'] == 'Русский Свет (Тверь)') & (df['Месяц'] == 'Январь')),
#         ('Distributor','Год', 'Месяц', 'Сумма отгрузки, с НДС, руб')]\
#         .groupby(['Distributor','Год', 'Месяц']).sum()

### Заполняем пустые поля для корректной агрегации данных

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3725625 entries, 0 to 1859142
Data columns (total 12 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   Код дистрибьютора              int64  
 1   Distributor                    object 
 2   Филиал                         object 
 3   ДДП                            object 
 4   Наименование товара            object 
 5   Название товара дистрибьютора  object 
 6   Артикул товара дистрибьютора   object 
 7   Сумма закупки, с НДС, руб      float64
 8   Сумма отгрузки, с НДС, руб     float64
 9   Год                            int64  
 10  Месяц                          object 
 11  Тип операции                   object 
dtypes: float64(2), int64(2), object(8)
memory usage: 369.5+ MB


In [24]:
df.fillna(0, inplace=True)

### Загружаем классификатор номенклатуры

In [26]:
# Загружаем классификатор номенклатуры
file_Class_Nom = 'C:/Users/lazarevnv/Desktop/НСИ, кубы и т.д/fac_КлассификаторНоменклатуры_07_10_2021.xlsx'

df_Class_Nom =  pd.ExcelFile(file_Class_Nom)

In [27]:
print(df_Class_Nom.sheet_names)

['Класс']


In [28]:
df_Class_Nom = df_Class_Nom.parse('Класс')

In [29]:
df_Class_Nom.shape

(78526, 37)

### Объединяем отчёт и классификатор

In [32]:
# df.tail(2)

In [33]:
df_ot = df[['Distributor', 'Артикул товара дистрибьютора', 'Название товара дистрибьютора', 'Год', 'Месяц', 'Сумма закупки, с НДС, руб','Сумма отгрузки, с НДС, руб']]
df_ot.head(2)

,Distributor,Артикул товара дистрибьютора,Название товара дистрибьютора,Год,Месяц,"Сумма закупки, с НДС, руб","Сумма отгрузки, с НДС, руб"
0,АВЕНТА (Брянск),MKP42-N-02-30-20,KREPTA 3 Корпус пластиковый КМПн 2/2 IP30 белы...,2020,Март,0.0,287.1
1,АВЕНТА (Брянск),MKP42-N-04-30-12,KREPTA 3 Корпус пластиковый КМПн 2/4 IP30 белы...,2020,Март,0.0,711.0


In [34]:
# df_Class_Nom.columns

In [35]:
# df_Class_Nom[['Направление',
#        'Направление Деятельности', 'Категория', 'Группа', 'Подгруппа',
#        'ПодПодГруппа', 'ПродактМенеджер', 'Артикул',
#        'НаименованиеНоменклатуры']].tail(5)

In [36]:
# Создаём урезанный классификтор по 2м полям
df_Class_1 = df_Class_Nom[['Артикул', 'Направление Деятельности']]

In [37]:
df_ot.shape

(3725625, 7)

In [38]:
# Добавляем в отчёт нужные поля из классификатора
df_ot_class = pd.merge(df_ot, df_Class_1, left_on = ['Артикул товара дистрибьютора'], right_on = ['Артикул'], how = 'left')

In [39]:
df_ot_class.shape

(3725625, 9)

In [87]:
# df_ot_class[['Год', 'Месяц', 'Сумма отгрузки, с НДС, руб']]\
#     .groupby(['Год','Месяц']).sum()

In [40]:
df_ot_class.head(2)

,Distributor,Артикул товара дистрибьютора,Название товара дистрибьютора,Год,Месяц,"Сумма закупки, с НДС, руб","Сумма отгрузки, с НДС, руб",Артикул,Направление Деятельности
0,АВЕНТА (Брянск),MKP42-N-02-30-20,KREPTA 3 Корпус пластиковый КМПн 2/2 IP30 белы...,2020,Март,0.0,287.1,MKP42-N-02-30-20,04. Корпуса
1,АВЕНТА (Брянск),MKP42-N-04-30-12,KREPTA 3 Корпус пластиковый КМПн 2/4 IP30 белы...,2020,Март,0.0,711.0,MKP42-N-04-30-12,04. Корпуса


In [41]:
df_ot_class['Направление Деятельности'].unique()

array(['04. Корпуса', '01. ОРЭ', '10. IEK - светотехника', '05. КНС',
       '30. Промавтоматизация', nan, '20. ITK', '40. Прочее'],
      dtype=object)

### Загружаем вспомогательный классификатор

In [42]:
file_klass_dop = 'C:/Users/lazarevnv/Desktop/НСИ, кубы и т.д/Доп_классификатор_основной.xlsx'
df_klass_dop = pd.ExcelFile(file_klass_dop)
print(df_klass_dop.sheet_names)

['Лист2', 'Направление', 'Направления_деятельности', 'Компании', 'Условия']


In [71]:
# df_dop = df_klass_dop.parse('Направления_деятельности')

In [43]:
df_dop_companies =  df_klass_dop.parse('Компании')

In [44]:
df_dop_conditions =  df_klass_dop.parse('Условия')

In [46]:
# df_dop_companies.head(200)

In [48]:
# df_dop_conditions.head(200)

In [49]:
# объединяем вспомогательные классификаторы в один
df_dop = pd.merge(df_dop_companies, df_dop_conditions, on = 'Контракт', how = 'left')

In [58]:
# df_dop[:60]

In [59]:
df_ot_class.columns

Index(['Distributor', 'Артикул товара дистрибьютора',
       'Название товара дистрибьютора', 'Год', 'Месяц',
       'Сумма закупки, с НДС, руб', 'Сумма отгрузки, с НДС, руб', 'Артикул',
       'Направление Деятельности'],
      dtype='object')

In [60]:
df_dop.columns

Index(['Компания', 'Контракт', 'Направления деятельности',
       'Направления_бонус'],
      dtype='object')

In [61]:
df_ot_class.shape, df_dop.shape

((3725625, 9), (208, 4))

In [62]:
# объединяем крайний отчет и вспомогательный классификатор
df_ot_class_dop = pd.merge(df_ot_class, df_dop, left_on = ['Distributor', 'Направление Деятельности'], right_on = ['Компания', 'Направления деятельности'], how = 'left')

In [63]:
df_ot_class_dop.columns

Index(['Distributor', 'Артикул товара дистрибьютора',
       'Название товара дистрибьютора', 'Год', 'Месяц',
       'Сумма закупки, с НДС, руб', 'Сумма отгрузки, с НДС, руб', 'Артикул',
       'Направление Деятельности', 'Компания', 'Контракт',
       'Направления деятельности', 'Направления_бонус'],
      dtype='object')

In [64]:
df_ot_class_dop.shape

(3725625, 13)

In [66]:
# df_ot_class_dop.tail()

In [68]:
# df_dop_companies['Компания'].tolist()

### Применяем conditions_companies когда нужно выбрать конкретные комании из всех

In [59]:
# conditions_companies = ((df_ot_class_dop['Компания'] == 'Название (Брянск)')
#                         |(df_ot_class_dop['Компания'] == 'Название (Воронеж)')
#                         |(df_ot_class_dop['Компания'] == 'Название (Барнаул)')
#                         |(df_ot_class_dop['Компания'] == 'Название (ВАЯК) (Курган)')
#                         |(df_ot_class_dop['Компания'] == 'Название (Москва)')
#                         |(df_ot_class_dop['Компания'] == 'Название (Краснодар)')
#                         |(df_ot_class_dop['Компания'] == 'Название (Москва)')
#                         |(df_ot_class_dop['Компания'] == 'Название (Пенза)')
#                         |(df_ot_class_dop['Компания'] == 'Название (Москва)')
#                         |(df_ot_class_dop['Компания'] == 'Название (Иркутск)')
#                         |(df_ot_class_dop['Компания'] == 'Название(Москва)')                
#                        )

In [70]:
# df_ot_class_dop.shape

In [65]:
# df_ot_class_dop = df_ot_class_dop.loc[conditions_companies]

In [71]:
# df_ot_class_dop.shape

In [69]:
# df_ot_class_dop.tail(2)

### Группируем итоговый отчёт

In [95]:
# Выбираем только необходимые поля
df_ot_class_dop = df_ot_class_dop[['Distributor', 'Год',
       'Месяц', 'Сумма закупки, с НДС, руб', 'Сумма отгрузки, с НДС, руб', 
       'Направление Деятельности','Направления_бонус']]

df_ot_class_dop.tail(2)

In [76]:
df_ot_class_dop.columns

Index(['Distributor', 'Год', 'Месяц', 'Сумма закупки, с НДС, руб',
       'Сумма отгрузки, с НДС, руб', 'Направление Деятельности',
       'Направления_бонус'],
      dtype='object')

### Заполняем пустые поля 0

In [77]:
# ещё раз на всякий случай заполняем пустые поля
df_ot_class_dop.fillna(0, inplace=True)

In [80]:
# df_ot_class_dop[['Distributor','Год', 'Месяц', 'Направления_бонус', 'Сумма отгрузки, с НДС, руб']]\
#     .groupby(['Distributor','Год','Направления_бонус']).sum()

In [81]:
# df_ot_class_dop.info()

### Группируем

In [82]:
df_ot_class_dop.columns

Index(['Distributor', 'Год', 'Месяц', 'Сумма закупки, с НДС, руб',
       'Сумма отгрузки, с НДС, руб', 'Направление Деятельности',
       'Направления_бонус'],
      dtype='object')

In [83]:
# группируем
df_result = df_ot_class_dop[['Distributor','Год', 'Месяц', 'Направления_бонус','Сумма закупки, с НДС, руб', 'Сумма отгрузки, с НДС, руб']].groupby(['Distributor','Год','Направления_бонус']).sum()

In [84]:
# сбрасываем индекс
df_result = df_result.reset_index()


In [87]:
# df_result

In [86]:
df_result.to_excel(r'C:/Users/lazarevnv/Desktop/Архив ДП/Запросы от РФО/Бонус годовой 4кв 21г запрос Козлова/df_result_19.01.xlsx', index=False)

### Вспомогательные вычисления для проверки

In [94]:
# собираем все корректировки данных в один метод при необходимости повторного расчета

def corrections(df):
    # Загружаем классификатор номенклатуры
    file_Class_Nom = 'C:/Users/lazarevnv/Desktop/НСИ, кубы и т.д/fac_КлассификаторНоменклатуры_07_10_2021.xlsx'
    df_Class_Nom =  pd.ExcelFile(file_Class_Nom)
    df_Class_Nom = df_Class_Nom.parse('Класс')
    
    #     формируем df_ot
    df_ot = df[['Distributor', 'Артикул товара дистрибьютора', 'Название товара дистрибьютора', 'Год', 'Месяц', 'Сумма закупки, с НДС, руб','Сумма отгрузки, с НДС, руб']]
    #     формируем df_Class_1
    df_Class_1 = df_Class_Nom[['Артикул', 'Направление Деятельности']]
    #     мержим, добавляем направление деятельности, получаем df_ot_class
    df_ot_class = pd.merge(df_ot, df_Class_1, left_on = ['Артикул товара дистрибьютора'], right_on = ['Артикул'], how = 'left')
    
    #     Загружаем вспомогательный классификатор
    file_klass_dop = 'C:/Users/lazarevnv/Desktop/НСИ, кубы и т.д/Доп_классификатор_основной.xlsx'
    df_klass_dop = pd.ExcelFile(file_klass_dop)
    
    #     формируем df_dop_companies 
    df_dop_companies =  df_klass_dop.parse('Компании')
    #     формируем df_dop_conditions
    df_dop_conditions =  df_klass_dop.parse('Условия')
    #     формируем df_dop
    df_dop = pd.merge(df_dop_companies, df_dop_conditions, on = 'Контракт', how = 'left')
    #    мержим дальше, получаем df_ot_class_dop 
    df_ot_class_dop = pd.merge(df_ot_class, df_dop, left_on = ['Distributor', 'Направление Деятельности'],\
                               right_on = ['Компания', 'Направления деятельности'], how = 'left')
    #     выбираем нужные поля
    df_ot_class_dop = df_ot_class_dop[['Distributor', 'Год',
       'Месяц', 'Сумма закупки, с НДС, руб', 'Сумма отгрузки, с НДС, руб', 
       'Направление Деятельности','Направления_бонус']]
    #     Заполняем пустые поля 0
    df_ot_class_dop.fillna(0, inplace=True)
    
    #    Группируем
    df_result = df_ot_class_dop[['Distributor','Год', 'Месяц', 'Направления_бонус',\
                                 'Сумма закупки, с НДС, руб', 'Сумма отгрузки, с НДС, руб']].\
                                groupby(['Distributor','Год','Направления_бонус']).sum()
    df_result = df_result.reset_index()
    
    return df_result

In [89]:
# df[['Год', 'Сумма отгрузки, с НДС, руб']]\
#     .groupby(['Год']).sum()

In [48]:
# df_help = df[['Год', 'Сумма отгрузки, с НДС, руб']]\
#     .groupby(['Год']).sum()
# df_help = df_help.reset_index()
# df_help.to_excel(r'C:/Users/lazarevnv/Desktop/Расчёты/Минимакс_1кв/Расчёт 07.05.21/help1.xlsx', index=False)

In [90]:
# df_ot[['Год', 'Сумма отгрузки, с НДС, руб']]\
#     .groupby(['Год']).sum()

In [60]:
# df_help = df_ot[['Год', 'Сумма отгрузки, с НДС, руб']]\
#     .groupby(['Год']).sum()
# df_help = df_help.reset_index()
# df_help.to_excel(r'C:/Users/lazarevnv/Desktop/Расчёты/Минимакс_1кв/Расчёт 07.05.21/help2.xlsx', index=False)

In [91]:
# df_ot_class[['Год', 'Сумма отгрузки, с НДС, руб']]\
#     .groupby(['Год']).sum()

In [62]:
# df_help = df_ot_class[['Год', 'Сумма отгрузки, с НДС, руб']]\
#     .groupby(['Год']).sum()
# df_help = df_help.reset_index()
# df_help.to_excel(r'C:/Users/lazarevnv/Desktop/Расчёты/Минимакс_1кв/Расчёт 07.05.21/help3.xlsx', index=False)

In [92]:
# df_ot_class_dop[['Год', 'Сумма отгрузки, с НДС, руб']]\
#     .groupby(['Год']).sum()

In [64]:
# df_help = df_ot_class_dop[['Год', 'Сумма отгрузки, с НДС, руб']]\
#     .groupby(['Год']).sum()
# df_help = df_help.reset_index()
# df_help.to_excel(r'C:/Users/lazarevnv/Desktop/Расчёты/Минимакс_1кв/Расчёт 07.05.21/help4.xlsx', index=False)

In [93]:
# df_result[['Год', 'Сумма отгрузки, с НДС, руб']]\
#     .groupby(['Год']).sum()